In [1]:
import datetime
import math
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

In [2]:
tweets = pd.read_csv("./data/tweets_all.csv", encoding="latin1")

In [3]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0


In [4]:
min_lat = tweets.LATITUDE.min()
min_long = tweets.LONGITUDE.min()

In [5]:
print(tweets.LATITUDE.max())
print(tweets.LATITUDE.min())
print(tweets.LONGITUDE.max())
print(tweets.LONGITUDE.min())

51.69515609741211
51.28390121459961
0.3412590026855469
-0.5209429860115051


In [6]:
lat_range = tweets.LATITUDE.max() - tweets.LATITUDE.min()
long_range = tweets.LONGITUDE.max() - tweets.LONGITUDE.min()

lat_bin_size = lat_range/9
long_bin_size= long_range/9

In [7]:
print(lat_range)
print(long_range)

0.4112548828125
0.862201988697052


In [8]:
print(lat_bin_size)
print(long_bin_size)

0.045694986979166664
0.04421548659984882


In [9]:
tweets["Space"] = tweets.LONGITUDE.apply(lambda long: (((long - min_long)//long_bin_size)+1))
tweets["Row"] = tweets.LATITUDE.apply(lambda lat: (9-((lat - min_lat)//lat_bin_size)+1))

In [10]:
tweets["Space"] = tweets["Space"].apply(lambda l: int(l))
tweets["Row"] = tweets["Row"].apply(lambda l: int(l))

In [ ]:
tweets["One"] = 1

In [12]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,long_bin,lat_bin,lat_long_bin
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0,-0.29986555301226103,51.375291188557945,"51.375291188557945,-0.29986555301226103"
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0,-0.16721909321271455,51.466681162516274,"51.466681162516274,-0.16721909321271455"
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0,-0.034572633413168075,51.329596201578774,"51.329596201578774,-0.034572633413168075"
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0,-0.034572633413168075,51.512376149495445,"51.512376149495445,-0.034572633413168075"
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0,0.18650479958607602,51.55807113647461,"51.55807113647461,0.18650479958607602"


Remove non-plaintext elements from tweets

In [ ]:
tweets = tweets[~((tweets["Row"] == 1) & (tweets["Space"] == 8 ))]
tweets = tweets[~((tweets["Row"] == 3) & (tweets["Space"] == 10 ))]

In [13]:
def parse_tweets(tweet):

    tweet = re.sub('@[^\s]+','',tweet).strip()
    tweet = re.sub('#[^\s]+','',tweet).strip()
    tweet = re.sub(r'https?:\/\/.*[\r\n]*','',tweet).strip()
    tweet = " ".join(re.findall("[a-zA-Z]+", tweet))
    
    return tweet

In [14]:
tweets["parsed_tweets"] = tweets.MESSAGETEXT.apply(lambda tweet: parse_tweets(tweet))

In [15]:
tweets.head()

,id,Name,X,Y,LONGITUDE,LATITUDE,MESSAGEDATE,USERID,USERSCREENNAME,MESSAGETEXT,LOCATION,HASHTAGS,ISRETWEET,long_bin,lat_bin,lat_long_bin,parsed_tweets
0,550226818624942080,NaN,-0.262150,51.391407,-0.262150,51.391407,31/12/2014 09:47:50,113918054,12Elbestreet,@julieo25 Not yet got tics for Liverpool waiti...,Kingston upon Thames; London,NaN,0,-0.29986555301226103,51.375291188557945,"51.375291188557945,-0.29986555301226103",Not yet got tics for Liverpool waiting till Fr...
1,550226823314178048,NaN,-0.149466,51.492560,-0.149466,51.492561,31/12/2014 09:47:52,465989904,alice_foster_95,So. Many. Accents #alicesadventureswithmegabus,London; England,alicesadventureswithmegabus,0,-0.16721909321271455,51.466681162516274,"51.466681162516274,-0.16721909321271455",So Many Accents
2,550226827944660992,NaN,-0.014891,51.354042,-0.014891,51.354042,31/12/2014 09:47:53,55872342,MissGee_Pee,If it doesn't make you happy by December 31st....,New Addington; London,NaN,0,-0.034572633413168075,51.329596201578774,"51.329596201578774,-0.034572633413168075",If it doesn t make you happy by December st le...
3,550226837537058816,NaN,0.006698,51.532020,0.006698,51.532021,31/12/2014 09:47:55,147266450,vinita_ramtri,@westfieldstrat thanks; and are you open tomor...,Stratford; London,NaN,0,-0.034572633413168075,51.512376149495445,"51.512376149495445,-0.034572633413168075",thanks and are you open tomorrow at all
4,550227214227505152,NaN,0.214417,51.582535,0.214417,51.582535,31/12/2014 09:49:25,518928608,CallumGordon7,@WestHam_Central but he is,Romford; London,NaN,0,0.18650479958607602,51.55807113647461,"51.55807113647461,0.18650479958607602",but he is


In [16]:
# tweets.groupby("datetime_to_nearest_hour")["parsed_tweets"].apply(lambda s: "%s" % ' '.join(s))
# tweets_by_hour = pd.DataFrame({"message_text_for_hour" : tweets.groupby("datetime_to_nearest_hour")["parsed_tweets"].apply(lambda s: "%s" % ' '.join(s))}).reset_index()

In [17]:
# need to get pos tag
class StemmerTokenizer(object):

    def __init__(self):
        self.stemmer = PorterStemmer()
        
    def __call__(self, doc):
        is_noun = lambda pos: pos[:2] == 'NN'
        doc = " ".join(re.findall("[a-zA-Z]+", doc))
        tokenized = nltk.word_tokenize(doc)
        nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)] 
        return [self.stemmer.stem(t) for t in word_tokenize(doc)]

In [18]:
len(tweets.lat_bin.unique())

10

In [19]:
len(tweets.long_bin.unique())

20

In [20]:
idf_by_lat_long_bin = {}

for lat_long_bin in tweets.lat_long_bin.unique():
    clf = TfidfVectorizer(tokenizer=StemmerTokenizer(), min_df=1, max_df=0.7, ngram_range=(1, 1), stop_words=set(stopwords.words('english')))
    try:
        clf.fit(tweets[tweets["lat_long_bin"] == lat_long_bin]["parsed_tweets"])
        word2idf = dict(zip(clf.get_feature_names(), clf.idf_))
        sorted_idf = sorted(word2idf.items(), key=lambda x: x[1], reverse=True)
    except ValueError:
        sorted_idf = []

    idf_by_lat_long_bin[lat_long_bin] = {
        "lat_long_bin": lat_long_bin,
        "sorted_idf": sorted_idf
    }
    
    print("Completed", lat_long_bin)

print("Finished!")

Completed 51.375291188557945,-0.29986555301226103
Completed 51.466681162516274,-0.16721909321271455
Completed 51.329596201578774,-0.034572633413168075
Completed 51.512376149495445,-0.034572633413168075
Completed 51.55807113647461,0.18650479958607602
Completed 51.649461110432945,-0.25565006641241217
Completed 51.603766123453774,-0.4325120128118075
Completed 51.466681162516274,-0.12300360661286575
Completed 51.512376149495445,-0.07878812001301694
Completed 51.603766123453774,0.14228931298622716
Completed 51.329596201578774,-0.12300360661286575
Completed 51.42098617553711,-0.21143457981256336
Completed 51.512376149495445,0.009642853186680789
Completed 51.512376149495445,-0.21143457981256336
Completed 51.603766123453774,-0.38829652621195865
Completed 51.42098617553711,0.0980738263863784
Completed 51.512376149495445,-0.12300360661286575
Completed 51.42098617553711,-0.34408103961210984
Completed 51.42098617553711,-0.16721909321271455
Completed 51.512376149495445,-0.4767274994116563
Completed

Completed 51.329596201578774,0.14228931298622716
Completed 51.55807113647461,0.3191512593856225
Completed 51.28390121459961,0.23072028618592488
Completed 51.649461110432945,0.3191512593856225
Completed 51.28390121459961,-0.034572633413168075
Completed 51.28390121459961,0.05385833978652954
Completed 51.512376149495445,0.27493577278577364
Completed 51.28390121459961,0.0980738263863784
Completed 51.69515609741211,0.18650479958607602
Completed 51.649461110432945,0.18650479958607602
Finished!


In [21]:
tweets.groupby(['lat_long_bin'])['id'] \
                             .count() \
                             .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) \
                             .head(5)

,lat_long_bin,count
82,"51.466681162516274,-0.16721909321271455",6977
102,"51.512376149495445,-0.16721909321271455",5672
81,"51.466681162516274,-0.12300360661286575",4978
101,"51.512376149495445,-0.12300360661286575",4063
100,"51.512376149495445,-0.07878812001301694",3818


In [28]:
idf_by_lat_long_bin["51.466681162516274,-0.16721909321271455"]["sorted_idf"][:1000]

[('aaaada', 9.157370441186774),
 ('aaaand', 9.157370441186774),
 ('abait', 9.157370441186774),
 ('abbestia', 9.157370441186774),
 ('abbiamo', 9.157370441186774),
 ('abbixashleigh', 9.157370441186774),
 ('abbracci', 9.157370441186774),
 ('abdullamandani', 9.157370441186774),
 ('abel', 9.157370441186774),
 ('abercrombi', 9.157370441186774),
 ('abg', 9.157370441186774),
 ('abianggg', 9.157370441186774),
 ('abiii', 9.157370441186774),
 ('abinelson', 9.157370441186774),
 ('abit', 9.157370441186774),
 ('abrazoooo', 9.157370441186774),
 ('abrupt', 9.157370441186774),
 ('absent', 9.157370441186774),
 ('abund', 9.157370441186774),
 ('acab', 9.157370441186774),
 ('acabado', 9.157370441186774),
 ('acaban', 9.157370441186774),
 ('academi', 9.157370441186774),
 ('acc', 9.157370441186774),
 ('acceler', 9.157370441186774),
 ('accident', 9.157370441186774),
 ('accompani', 9.157370441186774),
 ('accord', 9.157370441186774),
 ('acculturati', 9.157370441186774),
 ('accus', 9.157370441186774),
 ('accustom

In [23]:
idf_by_lat_long_bin["51.512376149495445,-0.16721909321271455"]

{'lat_long_bin': '51.512376149495445,-0.16721909321271455',
 'sorted_idf': [('aaaaaaaaaaaaaay', 8.950326176766623),
  ('aaaaaaah', 8.950326176766623),
  ('aaaaand', 8.950326176766623),
  ('aaaaaw', 8.950326176766623),
  ('aaaaw', 8.950326176766623),
  ('aaaggggeeessss', 8.950326176766623),
  ('aaah', 8.950326176766623),
  ('aaand', 8.950326176766623),
  ('aaww', 8.950326176766623),
  ('aazief', 8.950326176766623),
  ('abadia', 8.950326176766623),
  ('abar', 8.950326176766623),
  ('abba', 8.950326176766623),
  ('abd', 8.950326176766623),
  ('abe', 8.950326176766623),
  ('abebti', 8.950326176766623),
  ('abel', 8.950326176766623),
  ('abi', 8.950326176766623),
  ('abierto', 8.950326176766623),
  ('abil', 8.950326176766623),
  ('abisan', 8.950326176766623),
  ('aboard', 8.950326176766623),
  ('abrazo', 8.950326176766623),
  ('abrir', 8.950326176766623),
  ('abro', 8.950326176766623),
  ('abroad', 8.950326176766623),
  ('abu', 8.950326176766623),
  ('abuela', 8.950326176766623),
  ('abund'

In [24]:
idf_by_lat_long_bin["51.466681162516274,-0.12300360661286575"]

{'lat_long_bin': '51.466681162516274,-0.12300360661286575',
 'sorted_idf': [('aaaallliiivvvve', 8.819837166082237),
  ('aah', 8.819837166082237),
  ('aarika', 8.819837166082237),
  ('aaron', 8.819837166082237),
  ('abbi', 8.819837166082237),
  ('aben', 8.819837166082237),
  ('abi', 8.819837166082237),
  ('abiniz', 8.819837166082237),
  ('aboard', 8.819837166082237),
  ('abort', 8.819837166082237),
  ('abov', 8.819837166082237),
  ('abras', 8.819837166082237),
  ('abzyyyyyy', 8.819837166082237),
  ('ac', 8.819837166082237),
  ('acab', 8.819837166082237),
  ('acaban', 8.819837166082237),
  ('acabar', 8.819837166082237),
  ('acabei', 8.819837166082237),
  ('acabo', 8.819837166082237),
  ('academi', 8.819837166082237),
  ('acc', 8.819837166082237),
  ('accomplish', 8.819837166082237),
  ('accord', 8.819837166082237),
  ('accuraci', 8.819837166082237),
  ('ace', 8.819837166082237),
  ('acerto', 8.819837166082237),
  ('ach', 8.819837166082237),
  ('achei', 8.819837166082237),
  ('acho', 8.81

In [25]:
idf_by_lat_long_bin["51.512376149495445,-0.12300360661286575"]

{'lat_long_bin': '51.512376149495445,-0.12300360661286575',
 'sorted_idf': [('aaaaaargh', 8.616775808698373),
  ('aaaargh', 8.616775808698373),
  ('aan', 8.616775808698373),
  ('aap', 8.616775808698373),
  ('aaron', 8.616775808698373),
  ('abit', 8.616775808698373),
  ('abomin', 8.616775808698373),
  ('absinth', 8.616775808698373),
  ('absurd', 8.616775808698373),
  ('absurdli', 8.616775808698373),
  ('aburro', 8.616775808698373),
  ('acabei', 8.616775808698373),
  ('academi', 8.616775808698373),
  ('accetti', 8.616775808698373),
  ('accost', 8.616775808698373),
  ('accumul', 8.616775808698373),
  ('acimov', 8.616775808698373),
  ('acoust', 8.616775808698373),
  ('acreditar', 8.616775808698373),
  ('across', 8.616775808698373),
  ('action', 8.616775808698373),
  ('activ', 8.616775808698373),
  ('acton', 8.616775808698373),
  ('adah', 8.616775808698373),
  ('adair', 8.616775808698373),
  ('adapt', 8.616775808698373),
  ('address', 8.616775808698373),
  ('adha', 8.616775808698373),
  ('a

In [26]:
idf_by_lat_long_bin["51.512376149495445,-0.07878812001301694"]

{'lat_long_bin': '51.512376149495445,-0.07878812001301694',
 'sorted_idf': [('aaaaaaaaaaaaaaooo', 8.55459670666557),
  ('aaallll', 8.55459670666557),
  ('ababa', 8.55459670666557),
  ('abbiehennessey', 8.55459670666557),
  ('abdul', 8.55459670666557),
  ('abit', 8.55459670666557),
  ('abla', 8.55459670666557),
  ('abort', 8.55459670666557),
  ('absenc', 8.55459670666557),
  ('absenta', 8.55459670666557),
  ('absoframinglut', 8.55459670666557),
  ('abt', 8.55459670666557),
  ('abund', 8.55459670666557),
  ('acabei', 8.55459670666557),
  ('academi', 8.55459670666557),
  ('accident', 8.55459670666557),
  ('accorgo', 8.55459670666557),
  ('accumul', 8.55459670666557),
  ('accur', 8.55459670666557),
  ('acel', 8.55459670666557),
  ('ach', 8.55459670666557),
  ('achin', 8.55459670666557),
  ('acom', 8.55459670666557),
  ('acquaint', 8.55459670666557),
  ('action', 8.55459670666557),
  ('ada', 8.55459670666557),
  ('adakala', 8.55459670666557),
  ('adamseanryan', 8.55459670666557),
  ('adapt'